In [2]:
from random import randint, choices
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import re
from tqdm import tqdm, trange

import sqlite3
import pickle

ma = Okt()

In [3]:
with open('LDA_news_theta.pickle', 'rb') as fi:
    theta = pickle.load(fi)

with open('LDA_news_phi.pickle', 'rb') as fi:
    phi = pickle.load(fi)

with open('LDA_news_z.pickle', 'rb') as fi:
    z = pickle.load(fi)

with open('LDA_news_V.pickle', 'rb') as fi:
    V = pickle.load(fi)

In [6]:
K = 4
alpha = .1
beta = .1

In [7]:
# crawl to db
con = sqlite3.connect('./news_Crawl.db')
cur = con.cursor()

In [14]:
cur.execute('''
    SELECT id, content, category FROM news LIMIT 100;
''')
articles = cur.fetchall()

In [15]:
articles[0][1]

' (서울=연합뉴스) 조민정 박경준 기자 = 이태원 압사 참사 다음 날 윤석열 대통령 주재 회의에서 \'압사\'라는 단어를 쓰지 말 것과, \'이태원 사고\'라는 표현을 쓰라는 지시가 나왔다는 의혹이 제기됐다. 국회 \'용산 이태원 참사 진상규명과 재발 방지를 위한 국정조사 특별위원회\' 소속 더불어민주당 신현영 의원은 7일 참사 당시 보건복지부, 소방청, 소방본부 등의 관계자가 나눈 모바일 메신저 대화를 근거로 이같이 주장했다. 참사 다음 날인 10월 30일 오후 박향 보건복지부 공공보건정책관은 \'오늘 대통령주재 회의 결과 이태원압사사건을 \'압사\' 제외하시고 이태원 사고로 요청드려요\'라는 글을 올린다. 이후 한 관계자는 \'이태원 사고로 변경하겠습니다\'라고 답을 하고 박 정책관은 \'감사해요\'라는 메시지를 남긴다. 야당은 이 같은 대화 내용이 참사 대응에 주력했어야 할 정부가 그 책임을 축소하는 데 급급했다는 점을 보여준다고 지적했다. 신 의원은 "참담한 사고 진상을 밝히고 수습하기보다 10·29 참사의 본질을 교묘하게 왜곡하고 사실을 축소하려는 의도는 아니었는지 진상을 밝혀야 할 것"이라고 말했다. 같은 당 이수진 원내대변인은 이날 국회 브리핑에서 "압사임이 명백한데 \'압사\'를 빼라고 한 이유는 무엇인가"라며 "윤석열 정권이 참사 수습보다 책임 회피에 급급했음이 드러났다"고 말했다. 이어 "제대로 된 국정조사가 필요하다"며 "최소한의 책임 있는 조치조차 외면하는 정권을 더는 용납할 수 없다"고 했다. 참사 명칭을 둘러싼 이런 논란에 복지부 측은 기자들에게 보낸 메시지를 통해 "이태원 사고 명칭과 관련해서는 그간 여러 차례 정부 입장을 설명해 드린 바 있다"며 "추가로 답변할 내용은 없다"고 밝혔다. 행정안전부는 10월 31일에 발송한 \'이태원 사고 관련 지역 단위 합동분향소 설치 협조\' 공문에서 제단 중앙에 \'이태원 사고 사망자\'라고 쓰도록 안내했다. 그러나 한덕수 국무총리는 지난달 8일 국회 예산결산특별위원회 종합정책질의에서 "최근 

In [16]:
corpus = [ma.morphs(content[1]) for content in articles]

In [17]:
corpus[0]

['(',
 '서울',
 '=',
 '연합뉴스',
 ')',
 '조',
 '민정',
 '박경준',
 '기자',
 '=',
 '이태원',
 '압사',
 '참사',
 '다음',
 '날',
 '윤석열',
 '대통령',
 '주재',
 '회의',
 '에서',
 "'",
 '압사',
 "'",
 '라는',
 '단어',
 '를',
 '쓰지',
 '말',
 '것',
 '과',
 ',',
 "'",
 '이태원',
 '사고',
 "'",
 '라는',
 '표현',
 '을',
 '쓰라는',
 '지시',
 '가',
 '나왔다는',
 '의혹',
 '이',
 '제기',
 '됐다',
 '.',
 '국회',
 "'",
 '용산',
 '이태원',
 '참사',
 '진상',
 '규명',
 '과',
 '재발',
 '방지',
 '를',
 '위',
 '한',
 '국정조사',
 '특별',
 '위원회',
 "'",
 '소속',
 '더불어',
 '민주당',
 '신',
 '현영',
 '의원',
 '은',
 '7일',
 '참사',
 '당시',
 '보건복지부',
 ',',
 '소방청',
 ',',
 '소방본부',
 '등',
 '의',
 '관계자',
 '가',
 '나눈',
 '모바일',
 '메신저',
 '대화',
 '를',
 '근거',
 '로',
 '이같이',
 '주장',
 '했다',
 '.',
 '참사',
 '다음',
 '날인',
 '10월',
 '30일',
 '오후',
 '박향',
 '보건복지부',
 '공공',
 '보건',
 '정책',
 '관',
 '은',
 "'",
 '오늘',
 '대통령',
 '주재',
 '회의',
 '결과',
 '이태원',
 '압사',
 '사건',
 '을',
 "'",
 '압사',
 "'",
 '제외',
 '하시고',
 '이태원',
 '사고',
 '로',
 '요청',
 '드려요',
 "'",
 '라는',
 '글',
 '을',
 '올린다',
 '.',
 '이후',
 '한',
 '관계자',
 '는',
 "'",
 '이태원',
 '사고',
 '로',
 '변경',
 '하겠습니다',
 "'",
 

In [18]:
theta = dict() # 문서-토픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
phi = dict() # 단어픽 분포(빈도) => m번째 문서, k번째 토픽 => {m:{k:빈도}}
z = dict() # 문서-단어 토픽 할당 정보 => m번째 문서, l번째 단어 => {m:{l:k}}

V = list()
for m, d in enumerate(corpus):
    for l, w in enumerate(d):
        V.append(w.lower())
        if m not in z:
            z[m] = dict()
        if l not in z[m]:
            z[m][l] = 0
        z[m][l] = randint(1,K) # 토픽 할당

V = set(V)
i2w = {i: v for i, v in enumerate(V)}
w2i = {v: i for i, v in enumerate(V)}

for m, L in z.items():
    if m not in theta:
        theta[m] = dict()
    for l, t in L.items():
        if t not in theta[m]:
            theta[m][t] = 0
        theta[m][t] += 1

        v = w2i[corpus[m][l].lower()]

        if v not in phi:
            phi[v] = dict()
        if t not in phi[v]:
            phi[v][t] = 0
        phi[v][t] += 1

In [19]:
iteration = 100

for _ in trange(iteration):
    for m, d in enumerate(corpus):
        for l, w in enumerate(d):
            v = w2i[w.lower()]

            k = z[m][l] # gibb's 대상 삭제
            theta[m][k] -= 1
            phi[v][k] -= 1

            topic = list()
            for t in range(K):
                A = (phi[v].get(t+1,0) + beta) / sum([T.get(t+1,0)+beta for v, T in phi.items()])
                B = theta[m].get(t+1,0) + alpha
                topic.append(A*B)

            k = choices(range(K), weights=topic, k=1)[0]+1

            z[m][l] = k
            theta[m][k] = (theta[m][k] if k in theta[m] else 0) + 1
            phi[v][k] = (phi[v][k] if k in phi[v] else 0) + 1
    # print('{}%'.format(_))

  1%|          | 1/100 [01:44<2:52:06, 104.31s/it]

0%


  2%|▏         | 2/100 [03:24<2:46:36, 102.01s/it]

1%


  3%|▎         | 3/100 [05:17<2:52:31, 106.71s/it]

...

97%


 99%|█████████▉| 99/100 [2:40:02<01:37, 97.87s/it]

98%


100%|██████████| 100/100 [2:41:45<00:00, 97.05s/it]

99%


In [22]:
with open('LDA_news_theta.pickle', 'wb') as f:
    pickle.dump(theta, f)

with open('LDA_news_phi.pickle', 'wb') as f:
    pickle.dump(phi, f)

with open('LDA_news_z.pickle', 'wb') as f:
    pickle.dump(z, f)

with open('LDA_news_V.pickle', 'wb') as f:
    pickle.dump(V, f)


In [4]:
for l, T in phi.items():
    if T[1] > 0:
        print(i2w[l], T[1])

NameError: name 'i2w' is not defined